In [1]:
# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
from prettytable import PrettyTable

In [29]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [3]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [4]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [5]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [6]:

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).as_matrix()

In [7]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [8]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

C:\Users\hp\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [9]:
# Configuring a session
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [10]:
# Import Keras
from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

Using TensorFlow backend.


In [11]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

In [45]:
# Initializing parameters
"""
epochs = 30
batch_size = 16                  #this configuration is not improving our accuracy
n_hidden = 64                     accuracy is around 90.
drop_out = 0.5


epochs = 30
batch_size = 16                  # this is also not working well for us; accuracy 91.45 ~92%
n_hidden = 64
drop_out = 0.25


epochs = 30
batch_size = 32                  #this configuration seems to me as it's doing overfitting
n_hidden = 64                     accuracy increase to a certain point and then it decreases
drop_out = 0.25                     



epochs = 30                      # 92.4% 
batch_size = 64               
n_hidden = 256               
drop_out = 0.5


epochs = 30                     # performance decreases after 25/26 epochs
batch_size = 64                  accuracy = 88.6 whereas around 25/26 epchs accuracy was ~93%
n_hidden = 256                  
drop_out = 0.75


epochs = 35                     # 90.77
batch_size = 64                  
n_hidden = 256                  
drop_out = 0.6


epochs = 30                     # 92.23%
batch_size = 32                  
n_hidden = 256                  
drop_out = 0.6
"""

epochs = 35                     # 92.23%
batch_size = 32                  
n_hidden = 256                  
drop_out = 0.65

In [46]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [47]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

In [68]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


- Defining the Architecture of LSTM

In [48]:
import warnings
warnings.filterwarnings("ignore")

# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(drop_out))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

W0801 15:00:43.155395  5604 nn_ops.py:4224] Large dropout rate: 0.65 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 256)               272384    
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 1542      
Total params: 273,926
Trainable params: 273,926
Non-trainable params: 0
_________________________________________________________________


In [49]:
import warnings
warnings.filterwarnings("ignore")
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [50]:
import warnings
warnings.filterwarnings("ignore")

from datetime import datetime
start = datetime.now()

# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

print("Time taken : ", datetime.now() - start)

Train on 7352 samples, validate on 2947 samples
Epoch 1/35
7352/7352 [==============================] - 112s 15ms/step - loss: 1.3776 - acc: 0.3965 - val_loss: 1.3157 - val_acc: 0.4018
Epoch 2/35
7352/7352 [==============================] - 114s 16ms/step - loss: 1.1997 - acc: 0.4884 - val_loss: 1.1784 - val_acc: 0.4225
Epoch 3/35
7352/7352 [==============================] - 109s 15ms/step - loss: 1.2064 - acc: 0.4656 - val_loss: 1.2801 - val_acc: 0.3960
Epoch 4/35
7352/7352 [==============================] - 109s 15ms/step - loss: 0.9821 - acc: 0.5619 - val_loss: 0.9414 - val_acc: 0.5803
Epoch 5/35
7352/7352 [==============================] - 111s 15ms/step - loss: 0.7248 - acc: 0.6802 - val_loss: 0.6545 - val_acc: 0.7180
Epoch 6/35
7352/7352 [==============================] - 110s 15ms/step - loss: 0.5563 - acc: 0.7916 - val_loss: 0.4767 - val_acc: 0.8497
Epoch 7/35
7352/7352 [==============================] - 111s 15ms/step - loss: 0.3894 - acc: 0.8686 - val_loss: 0.3285 - val_acc: 

In [ ]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

In [22]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 1s 461us/step


In [23]:
score

[0.39372028857254565, 0.9019341703427214]

- With a simple 2 layer architecture we got 90.09% accuracy and a loss of 0.30
- We can further imporve the performace with Hyperparameter tuning

## LSTM model with 2 layers

In [97]:
"""                        this configuration gives accuracy of 90.84
epochs_m2 = 30
batch_size_m2= 32
n_hidden_layer1 = 128
n_hidden_layer2 =32
drop_out_1 = 0.2
drop_out_2 = 0.5


epochs_m2 = 30                    92.16
batch_size_m2= 32
n_hidden_layer1 = 128
n_hidden_layer2 =64
drop_out_1 = 0.2
drop_out_2 = 0.5



epochs_m2 = 30                   # 89.89
batch_size_m2= 8
n_hidden_layer1 = 32
n_hidden_layer2 =64
drop_out_1 = 0.5
drop_out_2 = 0.5


epochs_m2 = 30                   # 90.19
batch_size_m2= 64
n_hidden_layer1 = 32
n_hidden_layer2 =64
drop_out_1 = 0.5
drop_out_2 = 0.5


epochs_m2 = 60                   # 91.99
batch_size_m2= 64
n_hidden_layer1 = 32
n_hidden_layer2 =64
drop_out_1 = 0.5
drop_out_2 = 0.5


epochs_m2 = 120              # 91.99
batch_size_m2= 64
n_hidden_layer1 = 32
n_hidden_layer2 =64
drop_out_1 = 0.5
drop_out_2 = 0.5
"""

epochs_m2 = 50                  #92.16
batch_size_m2= 64
n_hidden_layer1 = 32
n_hidden_layer2 =64
drop_out_1 = 0.5
drop_out_2 = 0.5

In [98]:
from keras.layers.normalization import BatchNormalization as BNorm

# Initiliazing the sequential model
model2 = Sequential()
# Configuring the parameters
model2.add(LSTM(n_hidden_layer1, return_sequences=True, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model2.add(Dropout(drop_out_1))
# Adding batch normalization
model.add(BNorm())

model2.add(LSTM(n_hidden_layer2))
# Adding a dropout layer
model2.add(Dropout(drop_out_2))
# Adding a dense output layer with sigmoid activation
model2.add(Dense(n_classes, activation='sigmoid'))
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_28 (LSTM)               (None, 128, 32)           5376      
_________________________________________________________________
dropout_28 (Dropout)         (None, 128, 32)           0         
_________________________________________________________________
lstm_29 (LSTM)               (None, 64)                24832     
_________________________________________________________________
dropout_29 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 6)                 390       
Total params: 30,598
Trainable params: 30,598
Non-trainable params: 0
_________________________________________________________________


In [99]:
# Compiling the model
model2.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [100]:
import warnings
warnings.filterwarnings("ignore")

from datetime import datetime
start = datetime.now()

# Training the model
model2.fit(X_train,
          Y_train,
          batch_size=batch_size_m2,
          validation_data=(X_test, Y_test),
          epochs=epochs_m2)

print("Time taken : ", datetime.now() - start)

Train on 7352 samples, validate on 2947 samples
Epoch 1/50
7352/7352 [==============================] - 42s 6ms/step - loss: 1.3379 - acc: 0.4052 - val_loss: 1.1464 - val_acc: 0.4808
Epoch 2/50
7352/7352 [==============================] - 36s 5ms/step - loss: 1.0095 - acc: 0.5600 - val_loss: 0.9177 - val_acc: 0.5589
Epoch 3/50
7352/7352 [==============================] - 37s 5ms/step - loss: 0.8061 - acc: 0.6287 - val_loss: 0.8159 - val_acc: 0.5901
Epoch 4/50
7352/7352 [==============================] - 39s 5ms/step - loss: 0.7390 - acc: 0.6341 - val_loss: 0.7766 - val_acc: 0.6138
Epoch 5/50
7352/7352 [==============================] - 36s 5ms/step - loss: 0.6924 - acc: 0.6473 - val_loss: 0.7360 - val_acc: 0.6071
Epoch 6/50
7352/7352 [==============================] - 38s 5ms/step - loss: 0.6916 - acc: 0.6658 - val_loss: 0.7560 - val_acc: 0.6281
Epoch 7/50
7352/7352 [==============================] - 38s 5ms/step - loss: 0.6484 - acc: 0.6990 - val_loss: 0.8120 - val_acc: 0.6783
Epoch 8

In [83]:
score = model2.evaluate(X_test, Y_test)

2947/2947 [==============================] - 7s 2ms/step


In [84]:
score

[nan, 0.168306752629793]

## Conclusion


In classic machine learning models we got accuracy of around 96% and here we also tried to get the accuracy close to 96% by
* tuning the number of lstm units
* experimenting with drop out value 
* and by also adding a secnd hidden layer.

#### For our first model with single hidden layer I tried various configurations :

In [142]:
##################################### pretty table for model 1 with 1 lstm layer ####################################
number       = [1, 2, 3, 4]
epochs       = [30, 30, 30, 30]
batch_size   = [16, 16, 32, 32]
n_hidden     = [64, 64, 64, 128]
drop_out     = [0.25, 0.25, 0.25, 0.5]
accuracy     = [90.6, 91.45, 90.84, 92.81]

# Initializing prettytable # Adding columns 
ptable = PrettyTable()
ptable.add_column("Configuration",number)
ptable.add_column("Epochs", epochs)
ptable.add_column("Batch Size",batch_size)
ptable.add_column("Hidden Layer",n_hidden) 
ptable.add_column("Dropout",drop_out) 
ptable.add_column("Accuracy",accuracy) 
#Printing the Table
print(ptable)


+---------------+--------+------------+--------------+---------+----------+
| Configuration | Epochs | Batch Size | Hidden Layer | Dropout | Accuracy |
+---------------+--------+------------+--------------+---------+----------+
|       1       |   30   |     16     |      64      |   0.25  |   90.6   |
|       2       |   30   |     16     |      64      |   0.25  |  91.45   |
|       3       |   30   |     32     |      64      |   0.25  |  90.84   |
|       4       |   30   |     32     |     128      |   0.5   |  92.81   |
+---------------+--------+------------+--------------+---------+----------+


In [144]:
################################ pretty table for model 2 with 2 lstm layesr ##############################
number          = [1, 2]
epochs          = [30, 30]
batch_size      = [32, 32]
n_hidden_layer1 = [128, 128]
n_hidden_layer2 = [32, 64]
drop_out_1      = [0.2, 0.5]
drop_out_2      = [0.2, 0.5]
accuracy     = [90.84, 92.16]

# Initializing prettytable # Adding columns 
ptable = PrettyTable()
ptable.add_column("Configuration",number)
ptable.add_column("Epochs", epochs)
ptable.add_column("Batch Size",batch_size)
ptable.add_column("Hidden Layer",n_hidden_layer1)
ptable.add_column("Hidden Layer",n_hidden_layer2)
ptable.add_column("Dropout",drop_out_1)
ptable.add_column("Dropout",drop_out_2)
ptable.add_column("Accuracy",accuracy) 
#Printing the Table
print(ptable)


+---------------+--------+------------+--------------+--------------+---------+---------+----------+
| Configuration | Epochs | Batch Size | Hidden Layer | Hidden Layer | Dropout | Dropout | Accuracy |
+---------------+--------+------------+--------------+--------------+---------+---------+----------+
|       1       |   30   |     32     |     128      |      32      |   0.2   |   0.2   |  90.84   |
|       2       |   30   |     32     |     128      |      64      |   0.5   |   0.5   |  92.16   |
+---------------+--------+------------+--------------+--------------+---------+---------+----------+


* Note: One observation which I made while training our lstm models is that after training the model more than one time we get differnt values of accuracy i.e for same configuration we might get different accuracy values every time we train it over again.


So the results may improve if we train these models again with same configuration.
The final configurations for each model i.e model 1 and model 2 are good and we can get accuracy closer to 96%.